# Inteligencia Artificial Proyecto #2: Clasificación
## Integrantes
    - Luis Berrospi
    - Pedro Dominguez
    - Carlos Guerrero

### Lectura de datos

In [22]:
import pandas as pd
import numpy as np
import os

#Librerías de reducción de dimensiones
import pywt
import pywt.data

#Librerías para lectura/edición de imágenes
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import rgb2gray

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [23]:
df_test = pd.read_csv("..\data\sign_mnist_test.csv")
df_test_x = df_test.loc[:, "pixel1":"pixel784"]
df_test_y = df_test.label

df_train = pd.read_csv("..\data\sign_mnist_train.csv")
df_train_x = df_train.loc[:, "pixel1":"pixel784"]
df_train_y = df_train.label

### Reducción de dimensionalidad

In [24]:
print(df_train.shape)
print(df_test.shape)
random_state = 0
pca = make_pipeline(StandardScaler(), PCA(n_components=2, random_state=random_state))

(27455, 785)
(7172, 785)


In [25]:
def Haar(imagen, cortes):
  for i in range(cortes):
    imagen, (LH, HL, HH) = pywt.dwt2(imagen, 'haar')
  return imagen

def vectorizar(matrix):
  return matrix.flatten()

def proccess_images(route):
  dataset = []
  data_X = []
  data_Y = []
  for i, folder_name in enumerate(os.listdir(route)):

      for photo_name in os.listdir(route + folder_name):
        photo = imread(os.path.join(route + folder_name, photo_name))
        imagen = Haar(photo,2)
        features = vectorizar(imagen)
          
        data_X.append(features)
        data_Y.append((i, folder_name))
          
        features = np.append(features, i)
        dataset.append(features)
  return dataset, data_X, data_Y

### SVM

In [26]:
svm = svm.SVC(kernel='linear')
svm.fit(df_train_x,df_train_y)
svm_predicted = svm.predict(df_test_x)

#### Estimación del error

In [27]:
svm_success = 0.0
for i, val in enumerate(svm_predicted):
    if val == df_test_y[i]:
        svm_success += 1
print(svm_success/len(svm_predicted)*100)

78.16508644729504


#### Tabla de resumen

### KNN

In [28]:
knn = KNeighborsClassifier(n_neighbors=24)
knn.fit(df_train_x,df_train_y)
knn_predicted = knn.predict(df_test_x)

#### Estimación del error

In [29]:
knn_success = 0.0
for i, val in enumerate(knn_predicted):
    if val == df_test_y[i]:
        knn_success += 1
print(knn_success/len(knn_predicted)*100)

76.26882320133855


#### Tabla de resumen

### Decision tree

In [30]:
dt = tree.DecisionTreeClassifier()
dt.fit(df_train_x,df_train_y)
dt_predicted = dt.predict(df_test_x)

#### Estimación del error

In [31]:
dt_success = 0.0
for i, val in enumerate(dt_predicted):
    if val == df_test_y[i]:
        dt_success += 1
print(dt_success/len(dt_predicted)*100)

43.878973786949246


#### Tabla de resumen